In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re #For using Regex
!pip install langdetect
from langdetect import detect #to detect language
!pip install transformers
import transformers
from transformers import pipeline


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=90c384f34109e7787f6f8e3877c2c82b38a65b8194f748999df623b606baf6b4
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


2024-03-26 12:12:20.186348: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 12:12:20.186497: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 12:12:20.353852: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/Coding_Files_Prepared_for_NLP.xlsx


## Loading Excel to Pandas

In [5]:
xlsx_file = "/kaggle/input/Coding_Files_Prepared_for_NLP.xlsx"
xlsx_data = pd.read_excel(xlsx_file, sheet_name=None, header=0)
combined_df = xlsx_data['EverythingCombined']
m3_df = xlsx_data['M3_Quotation']
oporajita_df = xlsx_data['Oporajita_Quotation']
design_df = xlsx_data['Design_Quotation']


In [28]:
# columns = ['Participant', 'Description']
# trimmed_combined_df = combined_df[columns]
# trimmed_m3_df = m3_df[columns]
# trimmed_oporajita_df = oporajita_df[columns]
# trimmed_design_df = design_df[columns]
# print(trimmed_combined_df)
# print(trimmed_m3_df)
# print(trimmed_oporajita_df)
# print(trimmed_design_df)

## Cleaning Dataset

In [6]:
def clean_text(text):
    regex = r'-\s*P\d+$'
    cleaned_text = re.sub(regex, '', text)
    return cleaned_text

cleaned_combined_df = combined_df.copy()
cleaned_combined_df['Description'] = cleaned_combined_df['Description'].apply(clean_text)

cleaned_m3_df = m3_df.copy()
cleaned_m3_df['Description'] = cleaned_m3_df['Description'].apply(clean_text)

cleaned_oporajita_df = oporajita_df.copy()
cleaned_oporajita_df['Description'] = cleaned_oporajita_df['Description'].apply(clean_text)

cleaned_design_df = design_df.copy()
cleaned_design_df['Description'] = cleaned_design_df['Description'].apply(clean_text)

# print(cleaned_combined_df)
# print(cleaned_m3_df)
# print(cleaned_oporajita_df)
# print(cleaned_design_df)

In [7]:
output_file = 'cleaned_dataset.xlsx'
# Create an Excel writer object
with pd.ExcelWriter(output_file) as writer:
    # Write each DataFrame to a separate sheet in the Excel file
    cleaned_combined_df.to_excel(writer, sheet_name='Combined', index=False)
    cleaned_m3_df.to_excel(writer, sheet_name='M3', index=False)
    cleaned_oporajita_df.to_excel(writer, sheet_name='Oporajita', index=False)
    cleaned_design_df.to_excel(writer, sheet_name='Design', index=False)

# Notify user about the successful export
print(f"DataFrames exported to {output_file} successfully!")

DataFrames exported to cleaned_dataset.xlsx successfully!


## Converting Dataset To English


In [26]:
bn_to_en = pipeline("translation", model="csebuetnlp/banglat5_nmt_bn_en")

def convert_to_en(text):
    if detect(text) == 'bn':
        translation_result = bn_to_en(text)
        return translation_result[0]['translation_text']
    else:
        return text

translated_combined_df = cleaned_combined_df.copy()
translated_combined_df['Description'] = translated_combined_df['Description'].apply(convert_to_en)

translated_m3_df = cleaned_m3_df.copy()
translated_m3_df['Description'] = translated_m3_df['Description'].apply(convert_to_en)

translated_oporajita_df = cleaned_oporajita_df.copy()
translated_oporajita_df['Description'] = translated_oporajita_df['Description'].apply(convert_to_en)

translated_design_df = cleaned_design_df.copy()
translated_design_df['Description'] = translated_design_df['Description'].apply(convert_to_en)

# print(translated_combined_df)
# print(translated_m3_df)
# print(translated_oporajita_df)
# print(translated_design_df)

In [9]:
output_file = 'translated_dataset.xlsx'
# Create an Excel writer object
with pd.ExcelWriter(output_file) as writer:
    # Write each DataFrame to a separate sheet in the Excel file
    translated_combined_df.to_excel(writer, sheet_name='Combined', index=False)
    translated_m3_df.to_excel(writer, sheet_name='M3', index=False)
    translated_oporajita_df.to_excel(writer, sheet_name='Oporajita', index=False)
    translated_design_df.to_excel(writer, sheet_name='Design', index=False)

# Notify user about the successful export
print(f"DataFrames exported to {output_file} successfully!")

DataFrames exported to translated_dataset.xlsx successfully!


## Emotion Analysis

In [11]:
emotion_analysis = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions", top_k=3)
def analyze_emotion(text):
    result = emotion_analysis(text)
    label_1 = result[0][0]['label']
    score_1 = result[0][0]['score']
    label_2 = result[0][1]['label']
    score_2 = result[0][1]['score']
    label_3 = result[0][2]['label']
    score_3 = result[0][2]['score']
    return label_1, label_2, label_3, score_1, score_2, score_3


def apply_emotion_analysis(df):
    emotions = df['Description'].apply(lambda x: analyze_emotion(x))
    df['Emotion_Label_1'] = emotions.apply(lambda x: x[0])
    df['Emotion_Label_2'] = emotions.apply(lambda x: x[1])
    df['Emotion_Label_3'] = emotions.apply(lambda x: x[2])
    df['Emotion_Score_1'] = emotions.apply(lambda x: x[3])
    df['Emotion_Score_2'] = emotions.apply(lambda x: x[4])
    df['Emotion_Score_3'] = emotions.apply(lambda x: x[5])
    return df

# Apply emotion analysis to each DataFrame
em_combined_df = apply_emotion_analysis(translated_combined_df)
em_m3_df = apply_emotion_analysis(translated_m3_df)
em_oporajita_df = apply_emotion_analysis(translated_oporajita_df)
em_design_df = apply_emotion_analysis(translated_design_df)


# print(em_translated_combined_df)
# print(em_translated_m3_df)
# print(em_translated_oporajita_df)
# print(em_translated_design_df)

config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [12]:
output_file = 'calculated_dataset.xlsx'
# Create an Excel writer object
with pd.ExcelWriter(output_file) as writer:
    # Write each DataFrame to a separate sheet in the Excel file
    em_combined_df.to_excel(writer, sheet_name='Combined', index=False)
    em_m3_df.to_excel(writer, sheet_name='M3', index=False)
    em_oporajita_df.to_excel(writer, sheet_name='Oporajita', index=False)
    em_design_df.to_excel(writer, sheet_name='Design', index=False)

# Notify user about the successful export
print(f"DataFrames exported to {output_file} successfully!")

DataFrames exported to calculated_dataset.xlsx successfully!


In [30]:
concatenated_Combined_df = translated_combined_df.groupby('Participant')['Description'].apply(lambda x: ' '.join(x)).reset_index()
concatenated_m3_df = translated_m3_df.groupby('Participant')['Description'].apply(lambda x: ' '.join(x)).reset_index()
concatenated_oporajita_df = translated_oporajita_df.groupby('Participant')['Description'].apply(lambda x: ' '.join(x)).reset_index()
concatenated_design_df = translated_design_df.groupby('Participant')['Description'].apply(lambda x: ' '.join(x)).reset_index()

output_file = 'concatenated_dataset.xlsx'
# Create an Excel writer object
with pd.ExcelWriter(output_file) as writer:
    # Write each DataFrame to a separate sheet in the Excel file
    concatenated_Combined_df.to_excel(writer, sheet_name='Combined', index=False)
    concatenated_m3_df.to_excel(writer, sheet_name='M3', index=False)
    concatenated_oporajita_df.to_excel(writer, sheet_name='Oporajita', index=False)
    concatenated_design_df.to_excel(writer, sheet_name='Design', index=False)

# Notify user about the successful export
print(f"DataFrames exported to {output_file} successfully!")



DataFrames exported to concatenated_dataset.xlsx successfully!


In [32]:
# Apply emotion analysis to each DataFrame
em_concatanated_combined_df = apply_emotion_analysis(concatenated_Combined_df)
em_concatanated_m3_df = apply_emotion_analysis(concatenated_m3_df)
em_concatanated_oporajita_df = apply_emotion_analysis(concatenated_oporajita_df)
em_concatanated_design_df = apply_emotion_analysis(concatenated_design_df)


output_file = 'concatenated_calculated_dataset.xlsx'
# Create an Excel writer object
with pd.ExcelWriter(output_file) as writer:
    # Write each DataFrame to a separate sheet in the Excel file
   #em_concatenated_combined_df.to_excel(writer, sheet_name='Combined', index=False)
   em_concatenated_m3_df.to_excel(writer, sheet_name='M3', index=False)
   #em_concatenated_oporajita_df.to_excel(writer, sheet_name='Oporajita', index=False)
   em_concatenated_design_df.to_excel(writer, sheet_name='Design', index=False)

# Notify user about the successful export
print(f"DataFrames exported to {output_file} successfully!")


RuntimeError: The expanded size of the tensor (752) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 752].  Tensor sizes: [1, 514]